In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import matplotlib
from matplotlib.animation import FFMpegWriter
import string
from copy import deepcopy
import warnings 
from tqdm import tqdm
warnings.filterwarnings('ignore')
###################################################################################################
#############主程序

df = pd.read_csv('./trainData/originFranceData1.csv',sep = ';')
print("#################################################################")
print("df.head()\n",df.head())
#print("df.info()\n",df.info())
#print(df.describe())


print("#################################################################")
print("veh_lane.unique()\n",df.vehicle_lane.unique())
laneList = df.vehicle_lane.unique()
numLanes = len(df.vehicle_lane.unique())
print("numLanes \n",numLanes)
print("#################################################################")


########################################################################################
#######枚举每一个车道,获得红灯附近的车，以及车道的长度
for ilane,curLaneID in enumerate(df.vehicle_lane.unique()):#枚举每一个车道
#for ilane in tqdm(range(numLanes)):#枚举每一个车道
    curLaneID= laneList[ilane]
    
    #提取红灯时刻的车辆样本 LaneID:239331354_0 ; Time:4224.0 ; redID:VehicleFlowSouthToWest4.17
    #if curLaneID != '239331354_0': #for debug
    #   continue

    #1.获得车道长度
    redVehs = pd.DataFrame(columns=df.columns)  #建立空的二维数组，，并且列与数据库一致。
    print("建立空的红灯静止车辆二维数组 redVehs\n")
    
    vehInOneLane = df[df.vehicle_lane==curLaneID]#获得当前车道上所有车辆
    if vehInOneLane.empty == True:
        print("当前车道counter,车道总数,车道名:",ilane,numLanes,curLaneID)
        print("\n vehInOneLane.empty == True,当前车道没有车\n")
        continue
    maxLanePos =  max(vehInOneLane.vehicle_pos)#车道的长度
    print("当前车道counter,车道总数,车道名,车道长度：",ilane,numLanes,curLaneID,maxLanePos)
    ############################################################3
    #2.提取红灯车辆以及时刻
    vehInOneLane =vehInOneLane.sort_values(by='timestep_time',ascending=True)
   
    timeList = vehInOneLane.timestep_time.unique()
    lowSpeedFlag = 0
    for t in timeList:#枚举每个时间
        title = "2.提取红灯车辆以及时刻,LaneID:"+str(curLaneID)+";"+"Time:"+str(t)+";"
        #print(title)
        vehsAtTime = vehInOneLane[vehInOneLane.timestep_time == float(t)] 

        for index, veh in vehsAtTime.iterrows():#每一辆车
            #print(index,veh)
            vehX =  veh.vehicle_x
            vehY =  veh.vehicle_y
            vehVel = veh.vehicle_speed
            vehTime  = veh.timestep_time
            vehID = veh.vehicle_id
            vehicle_pos = veh.vehicle_pos
            if (vehVel <2/3.6) and (abs(vehicle_pos - maxLanePos)<5):#红色静止车辨别标准
                lowSpeedFlag = 1
                redVehs.loc[len(redVehs.index)] =veh
               
   
    
          

    if redVehs.empty == True:
        print("redVehs.empty == True")
        #input()
        continue
    print(redVehs.info())
    print(redVehs.head())

    #3.当有车处于红灯状态时而且车道上有多个车时,对每条道路进行分析
    speedFlagDict = {}
    for ired, redID in enumerate(redVehs.vehicle_id.unique()):
        
        print("3. 分析车道为%s,红灯静止车辆为%s时的情况" %(curLaneID,redID))
        redVehFocusTmp = redVehs[redVehs.vehicle_id == redID]#红灯状态的车辆ID
        timeList = redVehFocusTmp.timestep_time.values  
        # 我们认为红灯静止车在本车道的存在时间，就是红灯剩余时间，也就是
        #max(timeList)-min(timeList)
        
        
        #3.1 枚举当前道路上红灯状态下的时间内所有车，并获得最小速度。、
        #我们认为红灯静止在本车道的存在时间，就是红灯剩余时间
        locTmp1 = vehInOneLane.timestep_time >= min(timeList)
        locTmp2 = vehInOneLane.timestep_time <= (max(timeList)+20)#红灯后灯后的20秒也计算，最小绿灯时间15秒
        #locTmp3 = abs(maxLanePos - vehInOneLane.vehicle_pos)<100 ##距离红灯100米以内,红灯时间内车道内所有的车
        
        vehsAtTimeAndDist = vehInOneLane[locTmp1 & locTmp2]#距离红灯100米以内,红灯时间内车道内所有的车
        vehIDsAtTimeAndDist = vehsAtTimeAndDist.vehicle_id.unique()
        print("3.1 枚举当前道路上红灯状态下的时间内所有车，并获得最小速度\n",vehIDsAtTimeAndDist)
        for ii,idTmp  in enumerate(vehIDsAtTimeAndDist):
            vehTmp = vehsAtTimeAndDist[vehsAtTimeAndDist.vehicle_id== idTmp]#提取距离红灯100米以内,红灯时间内一部车的信息
            minSpeed = min(vehTmp.vehicle_speed.values)#距离红灯100米以内,红灯时间内一部车的是所有速度信息
            if minSpeed >= 35/3.6:
                speedFlag  = 4
            if minSpeed <=35/3.6 and minSpeed> 25/3.6:
                speedFlag  = 3
            if minSpeed <=25/3.6 and minSpeed> 15/3.6:
                speedFlag  = 2
            if minSpeed <=15/3.6 and minSpeed> 5/3.6:
                speedFlag  = 1
            if minSpeed <=5/3.6:
                speedFlag  = 0
            #注意时间分区
            speedFlagDict[idTmp] = speedFlag

        print("3.2 枚举当前道路上当前红灯静止车，红灯状态下的每个时间的每一辆车，并生成样本\n")
        
        samples3 = []#收集当前车道内对应红车有样本
        
        for t in timeList:#枚举红灯状态下的每个时间的每一辆车 
            
            
            locTmp1 =  vehInOneLane.timestep_time == t      
            locTmp2 = abs(maxLanePos - vehInOneLane.vehicle_pos)<100 #距离红灯100米以内

            vehsAtTime = vehInOneLane[locTmp1 & locTmp2] #符合车道，时间和距离限制
            
            if len(vehsAtTime.vehicle_id.unique()) == 1:  # 如果当前时间车辆只有一部车，统计忽略
                #print("vehsAtTime.vehicle_id.unique()) == 1,时间%d,redID:%s\n" %(t,redID))
                #print(vehsAtTime.vehicle_id.unique())
                #print("当前红色静止车的时间范围内的样本不要了\n")
                samples3 = []#当前红色静止车的时间范围内的样本不要了
                continue


            vehsAtTime =vehsAtTime.sort_values(by='vehicle_pos',ascending=False)
            samples = []#当前道路上红灯状态下当前红灯静止车的每个时间的每一辆车，并生成每个时刻样本
            counter = 0
            
            #3.2.1 枚举当前道路上红灯状态下的每个时间的每一辆车，并生成每个时刻样本
            title = "LaneID:"+str(curLaneID)+" ; "+"Time:"+str(t)+" ; "+"redID:"+redID
            print("3.2 举当前道路上红灯状态下当前红灯静止车的当前时间的每一辆车，并生成每个时刻样本\n",title)
            for rowindex, veh in vehsAtTime.iterrows():#每一辆车                  
                vehX =  veh.vehicle_x
                vehY =  veh.vehicle_y
                vehVel = veh.vehicle_speed
                vehTime  = veh.timestep_time
                vehID = veh.vehicle_id
                vehicle_Red_distane = maxLanePos - veh.vehicle_pos
                #print("vehID:",vehID," vehicle_Red_distane:",vehicle_Red_distane)
                samples2 = []
                if counter == 0 and vehID != redID:
                    print("第一部车不是红灯静止车，有错误，counter == 0,redID:",redID,"vehID",vehID)
                    samples3 = []#收集当前车道内对应红车有样本
                    break
                
                    

                if counter > 0:#生成用于机器学习的样本
                    avg_speed_lane = 60/3.6
                    max_speed_lane = 80/3.6
                    subject = [vehID,max(timeList) - t,#主车名，红灯剩余时间
                    vehicle_Red_distane,
                    vehVel,
                    avg_speed_lane,
                    vehicle_Red_distane/(vehVel+0.01),
                    vehicle_Red_distane/avg_speed_lane]
                    
                    samplesTmp = deepcopy(samples)
                    samplesTmp.extend([0, 0]*(20-counter))#主车前面的车（最大20车）车辆的状态

                    samples2 = deepcopy(subject)#当前样本：主车
                    samples2.extend(samplesTmp)#当前样本：主车+主车前面的车（最大20车）
                    samples2.extend([speedFlagDict[vehID]])#当前样本：主车+主车前面的车（最大20车）+speedFlag
                    samples3.append(samples2)#收集当前时间段内所有的样本
                    
                    #print("subject:",subject)
                    #print("samples:",samples)
                    #print("samplesTmp:",samplesTmp)
                    #print("samples2:",samples2)
                    
                        
                #当前道路上红灯状态下当前红灯静止车的每个时间的每一辆车，收集生成每个时刻样本,非机器学习的标准自定义样本     
                samples.extend([vehicle_Red_distane, vehVel])
                counter = counter+1
               
            

               
       



            #记录当前时刻排队情况
            #filename = '.\\franceRedData\\Platoon'+str(ilane)+redID+'+Time@'+str(t)+'.csv'
            #print(filename)
            #print("vehsAtTime.head()\n",vehsAtTime.head())
            #vehsAtTime.to_csv(filename)
        
        
        #当前车道，每个红灯车的所有时刻的样本
        name1 = ["vehID","redLightTime","distToRedLight","speed","laneAvgSpeed","arriveTime1","arriveTime2"]   
        name2 = ["vehPos_1","vehSpeed_1","vehPos_2","vehSpeed_2","vehPos_3","vehSpeed_3","vehPos_4","vehSpeed_4"] 
        name3 = ["vehPos_5","vehSpeed_5","vehPos_6","vehSpeed_6","vehPos_7","vehSpeed_7","vehPos_8","vehSpeed_8"]
        name4 = ["vehPos_9","vehSpeed_9","vehPos_10","vehSpeed_10","vehPos_11","vehSpeed_11","vehPos_12","vehSpeed_12"]
        name5 = ["vehPos_13","vehSpeed_13","vehPos_14","vehSpeed_14","vehPos_15","vehSpeed_15","vehPos_16","vehSpeed_16"]
        name6 = ["vehPos_17","vehSpeed_17","vehPos_18","vehSpeed_18","vehPos_19","vehSpeed_19","vehPos_20","vehSpeed_20"]
        headers = name1+name2+name3+name4+name5+name6+["speedFlag"]

        if samples3 != []:
            samplesTmp = pd.DataFrame(samples3,columns=headers)
            filename = './franceRedData/'+str(ilane)+'+'+redID+'.csv'
            samplesTmp.to_csv(filename,float_format='%.3f',index=0) 


###################################################################################
#将所有样本集合成一个CSV文件
import os
import pandas as pd
path = "./franceRedData/"

filelist = [path + i for i in os.listdir(path)]
dataset = pd.read_csv(filelist[0])

for tmpFile in filelist:
    if tmpFile.endswith(".csv"):
        #print(tmpFile)
        tmpDF = pd.read_csv(tmpFile)
        dataset = pd.concat([dataset,tmpDF],ignore_index=True,axis=0)
        

filename= "./trainData/"+"0_allSamples.csv" 
dataset.to_csv(filename,float_format='%.3f',index=0) 
        



In [11]:
########################重新优化

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import matplotlib
from matplotlib.animation import FFMpegWriter
import string
from copy import deepcopy
import warnings 
from tqdm import tqdm
warnings.filterwarnings('ignore')

###################################################################################################
#
def genSamples(vehInOneLane,redVehs):
    
    samplesAll = []#收集当前车道内对应红车所有样本


    for iRed, redID in enumerate(redVehs.vehicle_id.unique()):
        print(iRed,redID)
        
        redVehFocusTmp = redVehs[redVehs.vehicle_id == redID]#红灯状态的车辆ID
        timeList = redVehFocusTmp.timestep_time.values  

        vehInOneLane =vehInOneLane.sort_values(by='timestep_time',ascending=True)#提取持续的时间段
        maxLanePos =  max(vehInOneLane.vehicle_pos)#车道的长度
        
        
        
        for t in timeList:#枚举红灯状态下的每个时间的每一辆车 
            
      
            locTmp1 =  vehInOneLane.timestep_time == t      
            locTmp2 = abs(maxLanePos - vehInOneLane.vehicle_pos)<200 #距离红灯200米以内
            vehsAtTime = vehInOneLane[locTmp1 & locTmp2] #符合车道，时间和距离限制
                
                
            if len(vehsAtTime.vehicle_id.unique()) == 1:  # 如果当前时间车辆只有一部车，统计忽略
                #print("time",t,"len(vehsAtTime.vehicle_id.unique()) == 1")
                continue

           
            vehsAtTime =vehsAtTime.sort_values(by='vehicle_pos',ascending=False)
            counter = 0

            # 枚举当前道路上红灯状态下的每个时间的每一辆车，并生成每个时刻样本
            recordPerVeh = [];#记录每一辆车的[位置和速度]

            for rowindex, veh in vehsAtTime.iterrows():#每一辆车                  
                vehX =  veh.vehicle_x
                vehY =  veh.vehicle_y
                vehVel = veh.vehicle_speed
                vehTime  = veh.timestep_time
                vehID = veh.vehicle_id
                vehicle_Red_distane = maxLanePos - veh.vehicle_pos
                #print("vehID:",vehID," vehicle_Red_distane:",vehicle_Red_distane)

                if counter == 0 and vehID != redID:
                    #第一部车不是红灯静止车，有错误
                    print("counter == 0 and vehID != redID:",vehID,redID)
                    break

                recordPerVeh.extend([vehicle_Red_distane, vehVel])#记录每一辆车的[位置和速度]

                if counter > 0 and counter <19:#生成用于机器学习的样本,车辆不超过20
                    avg_speed_lane = 80/3.6
                    max_speed_lane = 80/3.6
                    
                    redTime = max(timeList) - t
                    realVehNum = counter
                    arrivalTimeDivRedTime = vehicle_Red_distane/max_speed_lane/(redTime+0.001)

                    #subject为主车样本
                    
                    subject = [vehID,max(timeList) - t,#主车名，红灯剩余时间
                    vehicle_Red_distane,
                    vehVel,
                    avg_speed_lane,
                    vehicle_Red_distane/(vehVel+0.01),
                    vehicle_Red_distane/avg_speed_lane,realVehNum,arrivalTimeDivRedTime]


                    #samplesTmp1为当前时刻主车前面的车（最大20车）车辆的状态
                    samplesTmp1 = deepcopy(recordPerVeh)
                    
                   
                    
                    samplesTmp1.extend([0, 0]*(19-counter))#主车前面的车（最大20车）车辆的状态


                    #samplesTmp2为当前时刻当前样本：主车+主车前面的车（最大20车）+speedFlag
                    samplesTmp2 = deepcopy(subject)#当前样本：主车
                    samplesTmp2.extend(samplesTmp1)#当前样本：主车+主车前面的车（最大20车）
                    samplesTmp2.extend([speedFlagDict[vehID]])


                    samplesAll.append(samplesTmp2)#收集所有的样本
                    
                counter = counter+1


            
            
   
    return samplesAll

###################################################################################################
#根据交通灯边红色静止车的特征，获得符合特征的红灯静止车

def extractRedVehs(vehInOneLane):
    
    redVehs = pd.DataFrame(columns=vehInOneLane.columns)  #建立空的二维数组，并且列与数据库一致。
    
    maxLanePos =  max(vehInOneLane.vehicle_pos)#车道的长度
    
    ####提取红灯车辆以及时刻
    vehInOneLane =vehInOneLane.sort_values(by='timestep_time',ascending=True)#提取持续的时间段
    timeList = vehInOneLane.timestep_time.unique()

    lowSpeedFlag = 0
    for t in timeList:#枚举每个时间，进行分析，获得红灯车辆
        
        vehsAtTime = vehInOneLane[vehInOneLane.timestep_time == float(t)]#当前时刻，当前车道上的所有车

        for index, veh in vehsAtTime.iterrows():#每一辆车
            #print(index,veh)
            vehX =  veh.vehicle_x
            vehY =  veh.vehicle_y
            vehVel = veh.vehicle_speed
            vehTime  = veh.timestep_time
            vehID = veh.vehicle_id
            vehicle_pos = veh.vehicle_pos
            if (vehVel <2/3.6) and (abs(vehicle_pos - maxLanePos)<3):#红色静止车辨别标准
                lowSpeedFlag = 1
                redVehs.loc[len(redVehs.index)] =veh
                
    return redVehs
    
###################################################################################################
#根据交通灯边红色静止车的特征，获得符合特征的所有其他车的最小速度
#认为其他车的最小速度是从计时开始到离开当前车道

def analyzingRedVehAtCurLane(redVehs,vehInOneLane,curLaneID):
    
    speedFlagDict = dict()
    maxLanePos =  max(vehInOneLane.vehicle_pos)#车道的长度
    
    for iRed, redID in enumerate(redVehs.vehicle_id.unique()):

        redVehFocusTmp = redVehs[redVehs.vehicle_id == redID]#红灯状态的车辆ID
        timeList = redVehFocusTmp.timestep_time.values  #红灯车的持续时间

        locTmp1 = vehInOneLane.timestep_time >= min(timeList)
        locTmp2 = vehInOneLane.timestep_time <= max(timeList)
        locTmp3 = abs(maxLanePos - vehInOneLane.vehicle_pos)<200 ##距离红灯200米以内,红灯时间内车道内所有的车

        #红灯时间内车道内所有的车,而且必须距离红灯200米以内，排除绿灯时在道路的车,核心数据1
        vehsAtTimeAndDist = vehInOneLane[locTmp1 & locTmp2 & locTmp3]
        vehIDsAtTimeAndDist = vehsAtTimeAndDist.vehicle_id.unique()#符合条件的所有车
        #print(vehsAtTimeAndDist.head(5))
        
        for ii,idTmp  in enumerate(vehIDsAtTimeAndDist):

            #提取符合ID的车，注意采用的是vehInOneLane，不是vehsAtTimeAndDist
            vehTmp =  vehInOneLane[vehInOneLane.vehicle_id==idTmp]

            #提取符合时间的车，  locTmp1 = vehInOneLane.timestep_time >= min(timeList)
            vehTmp = vehTmp[vehTmp.timestep_time >= min(timeList)]

            #提取符合位置的的车
            locTmp3 = abs(maxLanePos - vehTmp.vehicle_pos)<200
            vehTmp = vehTmp[locTmp3]

            minSpeed = min(vehTmp.vehicle_speed.values)#距离红灯100米以内,红灯时间内一部车的是所有速度信息
            if minSpeed >= 35/3.6:
                speedFlag  = 4
            if minSpeed <=35/3.6 and minSpeed> 25/3.6:
                speedFlag  = 3
            if minSpeed <=25/3.6 and minSpeed> 15/3.6:
                speedFlag  = 2
            if minSpeed <=15/3.6 and minSpeed> 5/3.6:
                speedFlag  = 1
            if minSpeed <=5/3.6:
                speedFlag  = 0
            #注意时间分区
            speedFlagDict[idTmp] = speedFlag


    return speedFlagDict



###################################################################################################
#############主程序

df = pd.read_csv('./trainData/originFranceData1.csv',sep = ';')

laneList = df.vehicle_lane.unique()#获得每一条道路
numLanes = len(df.vehicle_lane.unique())


########################################################################################
#######枚举每一个车道,获得红灯附近的车，以及车道的长度
for ilane,curLaneID in enumerate(df.vehicle_lane.unique()):#枚举每一个车道
  
    print("laneIndex is %d,nameID is %s" %(ilane,curLaneID))
    
    #if ilane<125:
    #    continue
    
    curLaneID= laneList[ilane]

    redVehs = pd.DataFrame(columns=df.columns)  #建立空的二维数组，并且列与数据库一致。
    
    vehInOneLane = df[df.vehicle_lane==curLaneID]#获得当前车道上所有车辆
    
    if vehInOneLane.empty == True:#当前车道没有车
        continue
        
    ###提取交通灯边的静止车    
    redVehs = extractRedVehs(vehInOneLane) 
    if redVehs.empty == True:#如果当前车道没有红灯车
         continue

            
    ####给出每辆车的最小速度        
    speedFlagDict = analyzingRedVehAtCurLane(redVehs,vehInOneLane,curLaneID)
    

    #并生成样本\n")
    samplesAll=genSamples(vehInOneLane,redVehs) 
        
    print("len(speedFlagDict):",len(speedFlagDict))
    print("len(redVehs):",len(redVehs))
    print("len(samplesAll):",len(samplesAll))
        
    #当前车道，每个红灯车的所有时刻的样本
    name1 = ["vehID","redLightTime","distToRedLight","speed","laneAvgSpeed","arriveTime1","arriveTime2","numStillVeh","ArrTimeDivRedTime"]   
    name2 = ["vehPos_1","vehSpeed_1","vehPos_2","vehSpeed_2","vehPos_3","vehSpeed_3","vehPos_4","vehSpeed_4"] 
    name3 = ["vehPos_5","vehSpeed_5","vehPos_6","vehSpeed_6","vehPos_7","vehSpeed_7","vehPos_8","vehSpeed_8"]
    name4 = ["vehPos_9","vehSpeed_9","vehPos_10","vehSpeed_10","vehPos_11","vehSpeed_11","vehPos_12","vehSpeed_12"]
    name5 = ["vehPos_13","vehSpeed_13","vehPos_14","vehSpeed_14","vehPos_15","vehSpeed_15","vehPos_16","vehSpeed_16"]
    name6 = ["vehPos_17","vehSpeed_17","vehPos_18","vehSpeed_18","vehPos_19","vehSpeed_19","vehPos_20","vehSpeed_20"]
    
    headers = name1+name2+name3+name4+name5+name6+["speedFlag"]

    if samplesAll != []:
        print(samplesAll[0])
        samplesTmp = pd.DataFrame(samplesAll,columns=headers)
        print(samplesTmp.info())
        filename = './franceRedData/'+str(ilane)+'+'+curLaneID+'.csv'
        samplesTmp.to_csv(filename,float_format='%.3f',index=0) 


###################################################################################
#将所有样本集合成一个CSV文件
import os
import pandas as pd
path = "./franceRedData/"

filelist = [path + i for i in os.listdir(path)]
dataset = pd.read_csv(filelist[0])

for tmpFile in filelist:
    if tmpFile.endswith(".csv"):
        #print(tmpFile)
        tmpDF = pd.read_csv(tmpFile)
        dataset = pd.concat([dataset,tmpDF],ignore_index=True,axis=0)
        

filename= "./trainData/"+"0_allSamples.csv" 
dataset.to_csv(filename,float_format='%.3f',index=0) 
        



laneIndex is 0,nameID is -1356_0
0 BusFlowEastWest0.0
1 BusFlowEastWest0.1
2 BusFlowEastWest0.2
3 BusFlowEastWest.0
4 BusFlowEastWest.1
5 BusFlowEastWest.2
6 BusFlowEastWest.3
7 BusFlowEastWest1.0
8 BusFlowEastWest1.2
9 BusFlowEastWest1.3
10 BusFlowEastWest2.0
11 BusFlowEastWest2.1
12 BusFlowEastWest2.3
13 BusFlowEastWest3.0
14 BusFlowEastWest3.1
15 BusFlowEastWest3.2
16 BusFlowEastWest3.4
17 BusFlowEastWest3.5
18 BusFlowEastWest4.0
19 BusFlowEastWest4.1
20 BusFlowEastWest4.2
21 BusFlowEastWest4.3
22 BusFlowEastWest4.4
23 BusFlowEastWest5.0
24 BusFlowEastWest5.1
25 BusFlowEastWest5.2
26 BusFlowEastWest5.3
27 BusFlowEastWest5.4
28 BusFlowEastWest5.5
29 BusFlowEastWest6.1
30 BusFlowEastWest6.2
len(speedFlagDict): 31
len(redVehs): 196
len(samplesAll): 0
laneIndex is 1,nameID is 28925528_0
0 VehicleFlowNorthToWest.0
1 VehicleFlowNorthToWest.2
2 VehicleFlowNorthToWest.3
3 VehicleFlowNorthToWest.5
4 VehicleFlowNorthToWest.6
5 VehicleFlowNorthToWest.8
6 VehicleFlowNorthToWest.9
7 VehicleFlowN

0 BusFlowSouthNorth0.0
1 VehicleFlowSouthToNorth.1
2 VehicleFlowSouthToWest.5
3 VehicleFlowSouthToWest.6
4 VehicleFlowSouthToWest.7
5 VehicleFlowSouthToNorth.4
6 VehicleFlowSouthToNorth.5
7 VehicleFlowSouthToWest.11
8 VehicleFlowSouthToWest.12
9 VehicleFlowSouthToNorth.8
10 VehicleFlowSouthToNorth.9
11 VehicleFlowSouthToNorth.10
12 VehicleFlowSouthToWest.18
13 VehicleFlowSouthToNorth.12
14 VehicleFlowSouthToWest.22
15 VehicleFlowSouthToNorth.15
16 VehicleFlowSouthToWest.26
17 VehicleFlowSouthToNorth.17
18 VehicleFlowSouthToNorth.18
19 VehicleFlowSouthToNorth.19
20 VehicleFlowSouthToNorth.21
21 VehicleFlowSouthToWest.35
22 VehicleFlowSouthToWest.38
23 VehicleFlowSouthToNorth.25
24 VehicleFlowSouthToNorth.26
25 VehicleFlowSouthToNorth.27
26 VehicleFlowSouthToWest.43
27 VehicleFlowSouthToWest.44
28 VehicleFlowSouthToWest.46
29 VehicleFlowSouthToWest1.0
30 BusFlowSouthNorth.0
31 VehicleFlowSouthToWest1.3
32 VehicleFlowSouthToWest1.4
33 VehicleFlowSouthToNorth1.4
34 VehicleFlowSouthToNorth1

laneIndex is 3,nameID is -240120122_0
0 BusFlowWestEast0.0
1 BusFlowWestEast0.1
2 BusFlowWestEast0.2
3 BusFlowWestEast.0
4 BusFlowWestEast.1
5 BusFlowWestEast.2
6 BusFlowWestEast.3
7 BusFlowWestEast1.0
8 BusFlowWestEast1.1
9 BusFlowWestEast1.2
10 BusFlowWestEast1.3
11 BusFlowWestEast2.0
12 BusFlowWestEast2.1
13 BusFlowWestEast2.2
14 BusFlowWestEast2.3
15 BusFlowWestEast3.1
16 BusFlowWestEast3.2
17 BusFlowWestEast3.4
18 BusFlowWestEast3.5
19 BusFlowWestEast4.0
20 BusFlowWestEast4.2
21 BusFlowWestEast4.3
22 BusFlowWestEast4.4
23 BusFlowWestEast5.0
24 BusFlowWestEast5.1
25 BusFlowWestEast5.2
26 BusFlowWestEast5.3
27 BusFlowWestEast5.4
28 BusFlowWestEast5.5
29 BusFlowWestEast6.1
len(speedFlagDict): 30
len(redVehs): 190
len(samplesAll): 0
laneIndex is 4,nameID is -1274_0
0 VehicleFlowEastToWest.3
1 VehicleFlowEastToWest.6
2 VehicleFlowEastToWest.7
3 VehicleFlowEastToWest.9
4 VehicleFlowEastToWest.10
5 VehicleFlowEastToWest.11
6 VehicleFlowEastToWest.12
7 VehicleFlowEastToWest.13
8 VehicleFl

30 VehicleFlowEastToWest1_0.2
31 VehicleFlowEastToWest1_0.5
32 VehicleFlowEastToWest1.6
33 VehicleFlowEastToWest1_0.8
34 VehicleFlowEastToWest1.9
35 VehicleFlowEastToWest1.10
36 VehicleFlowEastToWest1_1.9
37 VehicleFlowEastToWest1.13
38 VehicleFlowEastToWest1.15
39 VehicleFlowEastToWest1_1.14
40 VehicleFlowEastToWest1_1.16
41 VehicleFlowEastToWest1.19
42 VehicleFlowEastToNorth1.8
43 VehicleFlowEastToWest1.23
44 VehicleFlowEastToWest1_0.28
45 VehicleFlowEastToWest1_0.30
46 VehicleFlowEastToNorth1.11
47 VehicleFlowEastToWest1_0.33
48 VehicleFlowEastToWest1.33
49 VehicleFlowEastToWest1_0.36
50 VehicleFlowEastToWest1.36
51 VehicleFlowEastToWest1_0.38
52 VehicleFlowEastToWest1_1.37
53 VehicleFlowEastToWest1_0.42
54 VehicleFlowEastToWest1_1.40
55 VehicleFlowEastToNorth1.16
56 VehicleFlowEastToWest1_1.43
57 VehicleFlowEastToNorth1.17
58 VehicleFlowEastToWest1_1.46
59 VehicleFlowEastToNorth2.1
60 VehicleFlowEastToWest2_0.5
61 VehicleFlowEastToWest2_1.4
62 VehicleFlowEastToNorth2.4
63 VehicleFl

laneIndex is 6,nameID is -1280#0_0
0 VehicleFlowEastToWest_1.3
1 VehicleFlowEastToWest_1.46
2 VehicleFlowEastToWest1_1.15
3 VehicleFlowEastToWest1_1.17
4 VehicleFlowEastToWest2_1.42
5 VehicleFlowEastToWest2_1.44
6 VehicleFlowEastToWest2_1.46
7 VehicleFlowEastToWest3_1.11
8 VehicleFlowEastToWest3_1.46
9 VehicleFlowEastToWest4_1.36
10 VehicleFlowEastToWest6_1.7
11 VehicleFlowEastToWest6_1.46
12 VehicleFlowEastToWest5_1.8
len(speedFlagDict): 23
len(redVehs): 14
len(samplesAll): 11
['VehicleFlowEastToWest1_1.16', 0.0, 191.39, 13.36, 22.22222222222222, 14.31488406881077, 8.612549999999999, 1, 8612.55, 1.0199999999999818, 0.07, 191.39, 13.36, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 50 columns):
vehID                11 non-null object
redLightTime         11 non-null float64
distToRedLight       11 non-null float64
speed              

...

364 VehicleFlowWestToEast4_0.57
365 VehicleFlowWestToEast4.57
366 VehicleFlowWestToEast4.55
367 VehicleFlowWestToEast4.63
368 VehicleFlowWestToEast4_0.62
369 VehicleFlowWestToEast4.58
370 VehicleFlowSouthToEast4.22
371 VehicleFlowWestToEast4_0.64
372 VehicleFlowNorthToEast4.14
373 VehicleFlowWestToEast4.66
374 VehicleFlowSouthToEast4.24
375 VehicleFlowWestToEast4_0.70
376 VehicleFlowWestToEast4_0.72
377 VehicleFlowSouthToEast4.26
378 VehicleFlowWestToEast4.69
379 VehicleFlowNorthToEast4.15
380 VehicleFlowSouthToEast4.27
381 VehicleFlowWestToEast4_0.79
382 VehicleFlowWestToEast4.73
383 VehicleFlowNorthToEast4.16
384 VehicleFlowSouthToEast4.29
385 VehicleFlowSouthToEast4.31
386 VehicleFlowWestToEast4.79
387 VehicleFlowWestToEast4_0.85
388 VehicleFlowSouthToEast4.33
389 VehicleFlowWestToEast4.83
390 VehicleFlowWestToEast6_0.4
391 VehicleFlowWestToEast6_0.7
392 VehicleFlowNorthToEast6.3
393 VehicleFlowWestToEast6_0.6
394 VehicleFlowNorthToEast6.4
395 VehicleFlowWestToEast6_0.10
396 Vehicle

laneIndex is 126,nameID is :1214943548_3_2
0 VehicleFlowWestToNorth5.35
1 VehicleFlowWestToEast5_0.88
len(speedFlagDict): 3
len(redVehs): 3
len(samplesAll): 1
['VehicleFlowWestToEast5.82', 0.0, 7.71, 4.59, 22.22222222222222, 1.6760869565217393, 0.34695000000000004, 1, 346.95000000000005, 1.7199999999999998, 0.29, 7.71, 4.59, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 50 columns):
vehID                1 non-null object
redLightTime         1 non-null float64
distToRedLight       1 non-null float64
speed                1 non-null float64
laneAvgSpeed         1 non-null float64
arriveTime1          1 non-null float64
arriveTime2          1 non-null float64
numStillVeh          1 non-null int64
ArrTimeDivRedTime    1 non-null float64
vehPos_1             1 non-null float64
vehSpeed_1           1 non-null float64
vehPos_2             1 n

laneIndex is 130,nameID is 240120122_0
laneIndex is 131,nameID is -1348#1_0
laneIndex is 132,nameID is :2479387657_2_1
0 VehicleFlowEastToSouth.0
1 VehicleFlowNorthToEast.2
2 VehicleFlowNorthToEast.5
3 VehicleFlowEastToSouth.3
4 VehicleFlowNorthToEast.8
5 VehicleFlowEastToSouth.4
6 VehicleFlowNorthToEast.13
7 VehicleFlowNorthToSouth.13
8 VehicleFlowNorthToEast.20
9 VehicleFlowNorthToEast2.2
10 VehicleFlowNorthToEast2.10
11 VehicleFlowNorthToEast2.17
12 VehicleFlowEastToSouth2.6
13 VehicleFlowNorthToEast3.1
14 VehicleFlowEastToSouth3.5
15 VehicleFlowNorthToSouth4.8
16 VehicleFlowNorthToEast6.2
17 VehicleFlowNorthToEast6.3
18 VehicleFlowNorthToEast6.9
19 VehicleFlowNorthToEast5.2
20 VehicleFlowNorthToEast7.0
21 VehicleFlowNorthToSouth7.0
len(speedFlagDict): 24
len(redVehs): 204
len(samplesAll): 12
['VehicleFlowNorthToEast.1', 9.0, 6.37, 0.42, 22.22222222222222, 14.813953488372093, 0.28665, 1, 0.03184646150427731, 0.6100000000000003, 0.0, 6.37, 0.42, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

75 VehicleFlowWestToEast3_0.37
76 VehicleFlowWestToEast3.38
77 VehicleFlowSouthToEast3.29
78 VehicleFlowWestToEast3.45
79 VehicleFlowWestToEast3_0.52
80 VehicleFlowSouthToEast3.37
81 VehicleFlowWestToEast3.56
82 VehicleFlowWestToEast3.59
83 VehicleFlowSouthToEast3.46
84 VehicleFlowNorthToEast3.26
85 VehicleFlowWestToEast3_0.73
86 VehicleFlowSouthToEast4.0
87 VehicleFlowWestToEast4_0.5
88 VehicleFlowSouthToEast4.4
89 VehicleFlowSouthToEast4.5
90 VehicleFlowNorthToEast4.2
91 VehicleFlowWestToEast4.10
92 VehicleFlowWestToEast4.16
93 VehicleFlowWestToEast4.21
94 VehicleFlowWestToEast4_0.31
95 VehicleFlowSouthToEast4.16
96 VehicleFlowWestToEast4_0.38
97 VehicleFlowNorthToEast4.8
98 VehicleFlowWestToEast4_0.45
99 VehicleFlowWestToEast4_0.52
100 VehicleFlowWestToEast4.52
101 VehicleFlowNorthToEast4.13
102 VehicleFlowSouthToEast4.22
103 VehicleFlowWestToEast4.67
104 VehicleFlowWestToEast4_0.75
105 VehicleFlowSouthToEast4.29
106 VehicleFlowWestToEast4_0.82
107 VehicleFlowSouthToEast4.35
108 Veh

laneIndex is 142,nameID is -1340#0_0
laneIndex is 143,nameID is --1342_0
laneIndex is 144,nameID is -1326#0_0
laneIndex is 145,nameID is :-564_0_1
laneIndex is 146,nameID is -1326#0_1
laneIndex is 147,nameID is 105490271#1_1
laneIndex is 148,nameID is :2479049221_8_0
0 VehicleFlowWestToNorth5.51
len(speedFlagDict): 1
len(redVehs): 1
len(samplesAll): 0
laneIndex is 149,nameID is :2479049221_8_1
laneIndex is 150,nameID is -1326#1_0
laneIndex is 151,nameID is -1326#1_1
laneIndex is 152,nameID is :-702_1_0
laneIndex is 153,nameID is --1336#0_0
laneIndex is 154,nameID is -1326#2_0
laneIndex is 155,nameID is :1214943442_1_0
0 BusFlowNorthSouth3.1
1 VehicleFlowNorthToSouth6.2
2 VehicleFlowNorthToSouth6.7
len(speedFlagDict): 3
len(redVehs): 57
len(samplesAll): 0
laneIndex is 156,nameID is :1214943442_1_1
laneIndex is 157,nameID is -1326#2_1
laneIndex is 158,nameID is :1214943442_1_4
laneIndex is 159,nameID is :-808_0_0
laneIndex is 160,nameID is :431399476_0_0
laneIndex is 161,nameID is :31798